In [151]:
import numpy as np
import perceval as pcvl
import perceval.components as comp
import sympy as sp
from perceval.components import catalog
from perceval.converters import QiskitConverter
from perceval.algorithm import Analyzer, Sampler
from perceval.rendering.circuit import SymbSkin, PhysSkin
import random
import math


In [152]:
def generate_random_radians(length):
    radians = []
    for _ in range(length):
        radians.append(comp.PS(random.uniform(0, 2*math.pi)))
    return radians



In [153]:
def build_discriminator_circuit (circuit, phi_value_list): 

    phi_index = 0

    list = [0,1,2,4,5,6]
    for i in list:
        circuit.add(i, phi_value_list[phi_index])
        phi_index += 1

    circuit.add((0, 1), comp.BS())
    circuit.add((2, 3), comp.BS())
    circuit.add((4, 5), comp.BS())
    circuit.add((6, 7), comp.BS())

    # list lines with phase shift
    list = [0,2,4,6]
    for i in list:
        circuit.add(i, phi_value_list[phi_index])
        phi_index += 1


    circuit.add((0, 1), comp.BS())
    circuit.add((2, 3), comp.BS())
    circuit.add((4, 5), comp.BS())
    circuit.add((6, 7), comp.BS())

    circuit.add((1, 2), comp.BS())
    circuit.add((5, 6), comp.BS())

    # list lines with phase shift
    list = [1,5]
    for i in list:
        circuit.add(i, phi_value_list[phi_index])
        phi_index += 1

    circuit.add((1, 2), comp.BS())
    circuit.add((5, 6), comp.BS())


    pcvl.pdisplay(circuit, skin=SymbSkin())
    
    return circuit


In [154]:
def build_generator_and_discriminator_circuit (circuit, generator_phi_value_list, discriminator_phi_value_list): 
    
    phi_index = 0

    # list lines with phase shift
    list = [1,2,3,5,6,7]
    for i in list:
        circuit.add(i, generator_phi_value_list[phi_index])
        phi_index += 1

    circuit.add((1, 2), comp.BS())
    circuit.add((5, 6), comp.BS())

    # list lines with phase shift
    list = [1,5]
    for i in list:
        circuit.add(i, generator_phi_value_list[phi_index])
        phi_index += 1

    circuit.add((1, 2), comp.BS())
    circuit.add((5, 6), comp.BS())

    circuit.add((0, 1), comp.BS())
    circuit.add((2, 3), comp.BS())
    circuit.add((4, 5), comp.BS())
    circuit.add((6, 7), comp.BS())

    # list lines with phase shift
    list = [0,2,4,6]
    for i in list:
        circuit.add(i, generator_phi_value_list[phi_index])
        phi_index += 1

    circuit.add((0, 1), comp.BS())
    circuit.add((2, 3), comp.BS())
    circuit.add((4, 5), comp.BS())
    circuit.add((6, 7), comp.BS())

    # list lines with phase shift
    list = [1,5]
    for i in list:
        circuit.add(i, generator_phi_value_list[phi_index])
        phi_index += 1

    circuit.add((1, 2), comp.BS())
    circuit.add((5, 6), comp.BS())

    # list lines with phase shift
    list = [1,5]
    for i in list:
        circuit.add(i, generator_phi_value_list[phi_index])
        phi_index += 1
        
    circuit.add((1, 2), comp.BS())
    circuit.add((5, 6), comp.BS())

    # list lines with phase shift
    list = [0,2,4,6]
    for i in list:
        circuit.add(i, generator_phi_value_list[phi_index])
        phi_index += 1

    circuit.add((0, 1), comp.BS())
    circuit.add((2, 3), comp.BS())
    circuit.add((4, 5), comp.BS())
    circuit.add((6, 7), comp.BS())

    # list lines with phase shift
    list = [0,2,4,6]
    for i in list:
        circuit.add(i, generator_phi_value_list[phi_index])
        phi_index += 1

    circuit.add((0, 1), comp.BS())
    circuit.add((2, 3), comp.BS())
    circuit.add((4, 5), comp.BS())
    circuit.add((6, 7), comp.BS())

    list = [0,1,2,4,5,6]
    for i in list:
        circuit.add(i, generator_phi_value_list[phi_index])
        phi_index += 1

    # adds the discriminator to this circuit
    circuit = build_discriminator_circuit(circuit, discriminator_phi_value_list)

    pcvl.pdisplay(circuit, skin=SymbSkin())

    return circuit

In [155]:
# builds the generator+discriminator circuit and the discriminator circuit with given randomized phi values, outputting them in a 2-element list of circuits

def build_circuits (generator_phi_value_list, discriminator_phi_value_list):
    
    generator_and_discriminator_circuit = pcvl.Circuit(8)
    discriminator_circuit = pcvl.Circuit(8)

    generator_and_discriminator_circuit = build_generator_and_discriminator_circuit(generator_and_discriminator_circuit, generator_phi_value_list, discriminator_phi_value_list)
    discriminator_circuit = build_discriminator_circuit(discriminator_circuit, discriminator_phi_value_list)

    return [generator_and_discriminator_circuit, discriminator_circuit]
    


In [156]:
# get probabilities of the provided and true fock states from their corresponding circuits

provided_fock_state = pcvl.StateVector("|1,0,0,0,1,0,0,0>") + pcvl.StateVector("|0,1,0,0,0,1,0,0>") + pcvl.StateVector("|0,0,1,0,0,0,1,0>") + pcvl.StateVector("|0,0,0,1,0,0,0,1>")
true_fock_state = pcvl.StateVector("|1,0,0,0,0,1,0,0>") + pcvl.StateVector("|0,1,0,0,0,0,1,0>") + pcvl.StateVector("|0,0,1,0,0,0,0,1>") + pcvl.StateVector("|0,0,0,1,1,0,0,0>")

## build the circuits with randomized phi values

### randomized phi values
generator_phi_value_list = generate_random_radians(30)
discriminator_phi_value_list = generate_random_radians(12)

### build the circuits
circuit_list = build_circuits(generator_phi_value_list, discriminator_phi_value_list)
gen_and_disc_circ = circuit_list[0]
disc_circ = circuit_list[1]



## calculate the probability of the generator+discriminator circuit with generated fock state
gen_and_disc_circ_p = pcvl.Processor("SLOS", gen_and_disc_circ)
gen_and_disc_circ_p.with_input(provided_fock_state)
sampler1 = pcvl.algorithm.Sampler(gen_and_disc_circ_p)

gen_and_disc_probability = sampler1.probs()['results'].__getitem__(pcvl.BasicState([0,0,1,0,0,0,1,0]))
print("Probability with provided fock state and generator + discriminator circuit:", gen_and_disc_probability)



## calculate the probability of the discriminator circuit with true fock state
disc_circ_p = pcvl.Processor("SLOS", disc_circ)
disc_circ_p.with_input(true_fock_state)
sampler2 = pcvl.algorithm.Sampler(disc_circ_p)

disc_probability = sampler2.probs()['results'].__getitem__(pcvl.BasicState([0,0,1,0,0,0,1,0]))
print("Probability with true fock state and discriminator circuit:", disc_probability)



Probability with provided fock state and generator + discriminator circuit: 0.07576972741571056
Probability with true fock state and discriminator circuit: 0.027036187145230862


In [157]:
# loss function

loss_function = abs(gen_and_disc_probability) - abs(disc_probability)
print("Loss value:",loss_function)

Loss value: 0.0487335402704797
